In [1]:
# Taipy Dashboard - Suicide Rate Analysis


import pandas as pd
from taipy import Gui
import numpy as np
from scipy import stats
import warnings
import datetime as dt

warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression

In [2]:
!pip install --upgrade taipy


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import websocket

def on_message(ws, message):
    print(message)

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("WebSocket closed")

def on_open(ws):
    print("WebSocket opened")
    ws.send("Hello, World!")

try:
    ws = websocket.WebSocketApp("ws://echo.websocket.org/",
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close,
                                on_open=on_open)
    ws.run_forever()
    print("WebSocket connection established!")
except:
    print("WebSocket connection failed!")


WebSocket connection failed!


In [4]:
# after cleaning
data = pd.read_csv("D:/Jupyter Notebook/taipy/data_taipy/suicide.csv")

## Worldwide metrics

### Worldwide suicide by Gender 

In [5]:
world_suicide_sex = (
    data.groupby(["year", "sex"]).agg({"suicides/100k pop": "mean"}).reset_index()
)
world_suicide_sex["suicides/100k pop"] = round(
    world_suicide_sex["suicides/100k pop"], 2
)

In [6]:
df_sex = world_suicide_sex.set_index(["year", "sex"]).unstack()
df_sex.columns = [f"{i}_{j}" for i, j in df_sex.columns]

In [7]:
world_suicide_sex = df_sex.reset_index()

In [8]:
world_suicide_sex.tail()

,year,suicides/100k pop_female,suicides/100k pop_male
31,2016,4.82,22.02
32,2017,3.33,7.43
33,2018,3.45,7.44
34,2019,3.25,6.20
35,2020,0.10,0.36


In [9]:
## Avg suicide per 100k of 2 genders in 35 years

avg_sui_man = round(
    sum(world_suicide_sex["suicides/100k pop_male"]) / len(world_suicide_sex["year"]), 2
)
avg_sui_woman = round(
    sum(world_suicide_sex["suicides/100k pop_female"]) / len(world_suicide_sex["year"]),
    2,
)
print("man avg:", avg_sui_man, "\nwoman avg:", avg_sui_woman)

man avg: 18.55 
woman avg: 5.12


In [10]:
gender_pct = pd.DataFrame(
    {
        "Gender": ["Female", "Male"],
        "% Suicides": [
            round(
                sum(world_suicide_sex["suicides/100k pop_female"])
                / sum(
                    world_suicide_sex["suicides/100k pop_female"]
                    + world_suicide_sex["suicides/100k pop_male"]
                ),
                2,
            ),
            round(
                sum(world_suicide_sex["suicides/100k pop_male"])
                / sum(
                    world_suicide_sex["suicides/100k pop_female"]
                    + world_suicide_sex["suicides/100k pop_male"]
                ),
                2,
            ),
        ],
    }
)
gender_pct

,Gender,% Suicides
0,Female,0.22
1,Male,0.78


### Worldwide suicides by Age

In [11]:
world_suicide_age = (
    data.groupby(["year", "age"]).agg({"suicides/100k pop": "mean"}).reset_index()
)
world_suicide_age["suicides/100k pop"] = round(
    world_suicide_age["suicides/100k pop"], 2
)

In [12]:
df_age = world_suicide_age.set_index(["year", "age"]).unstack()
df_age.columns = [f"{i}_{j}" for i, j in df_age.columns]
world_suicide_age = df_age.reset_index()

In [13]:
world_suicide_age["suicides/100k pop_5-14 years"] = world_suicide_age[
    "suicides/100k pop_5-14 years"
].replace(np.nan, 0)
world_suicide_age.head()

,year,suicides/100k pop_15-24 years,suicides/100k pop_25-34 years,suicides/100k pop_35-54 years,suicides/100k pop_5-14 years,suicides/100k pop_55-74 years,suicides/100k pop_75+ years
0,1985,8.43,10.01,12.40,0.54,15.09,24.48
1,1986,8.15,10.59,11.70,0.42,13.97,23.71
2,1987,7.49,10.96,12.38,0.48,14.95,23.60
3,1988,8.75,12.41,13.18,0.48,15.84,25.60
4,1989,9.16,12.18,13.58,0.55,16.00,25.80


In [14]:
column = world_suicide_age.columns.tolist()
avg_sui_age = {}
for i in column[1:7]:
    avg_sui_age[i] = round(
        sum(world_suicide_age[i]) / len(world_suicide_age["year"]), 2
    )

In [15]:
avg_sui_age

{'suicides/100k pop_15-24 years': 8.12,
 'suicides/100k pop_25-34 years': 11.14,
 'suicides/100k pop_35-54 years': 13.97,
 'suicides/100k pop_5-14 years': 0.54,
 'suicides/100k pop_55-74 years': 15.02,
 'suicides/100k pop_75+ years': 21.82}

In [16]:
## Avg suicide per 100k of each age group in 35 years
avg_sui_age = pd.DataFrame({"age": avg_sui_age.keys(), "value": avg_sui_age.values()})
avg_sui_age

,age,value
0,suicides/100k pop_15-24 years,8.12
1,suicides/100k pop_25-34 years,11.14
2,suicides/100k pop_35-54 years,13.97
3,suicides/100k pop_5-14 years,0.54
4,suicides/100k pop_55-74 years,15.02
5,suicides/100k pop_75+ years,21.82


In [17]:
sum_all_age = (
    sum(world_suicide_age[column[1]])
    + sum(world_suicide_age[column[2]])
    + sum(world_suicide_age[column[3]])
    + sum(world_suicide_age[column[4]])
    + sum(world_suicide_age[column[5]])
    + sum(world_suicide_age[column[6]])
)

In [18]:
## % suicide rate compared to the total of each age group
age_pct = pd.DataFrame(
    {
        "Age": [
            "Group 5-14",
            "Group 15-24",
            "Group 25-34",
            "Group 35-54",
            "Group 55-74",
            "Group 75+",
        ],
        "% Suicides": [
            round(
                sum(world_suicide_age["suicides/100k pop_5-14 years"]) / sum_all_age, 2,
            ),
            round(
                sum(world_suicide_age["suicides/100k pop_15-24 years"]) / sum_all_age,
                2,
            ),
            round(
                sum(world_suicide_age["suicides/100k pop_25-34 years"]) / sum_all_age,
                2,
            ),
            round(
                sum(world_suicide_age["suicides/100k pop_35-54 years"]) / sum_all_age,
                2,
            ),
            round(
                sum(world_suicide_age["suicides/100k pop_55-74 years"]) / sum_all_age,
                2,
            ),
            round(
                sum(world_suicide_age["suicides/100k pop_75+ years"]) / sum_all_age, 2,
            ),
        ],
    }
)
age_pct

,Age,% Suicides
0,Group 5-14,0.01
1,Group 15-24,0.12
2,Group 25-34,0.16
3,Group 35-54,0.20
4,Group 55-74,0.21
5,Group 75+,0.31


### Worldwide suicides by generation

In [19]:
world_suicide_generation = (
    data.groupby(["year", "generation"])
    .agg({"suicides/100k pop": "mean"})
    .reset_index()
)
world_suicide_generation["suicides/100k pop"] = round(
    world_suicide_generation["suicides/100k pop"], 2
)

df_generation = world_suicide_generation.set_index(["year", "generation"]).unstack()
df_generation.columns = [f"{i}_{j}" for i, j in df_generation.columns]
world_suicide_generation = df_generation.reset_index()

### Avg suicides per 100k in each year

In [20]:
world_suicide = (
     data.groupby("year")
    .agg({"suicides/100k pop": "mean"})
    .rename(columns={"suicides/100k pop": "Total suicides per 100k"})
    .reset_index()
)
world_suicide["Total suicides per 100k"] = round(world_suicide["Total suicides per 100k"], 2)

world_suicide.tail()


,year,Total suicides per 100k
31,2016,13.42
32,2017,5.38
33,2018,5.44
34,2019,4.73
35,2020,0.23


In [21]:
## Avg suicides in 35 years
avg_rate = round(
    sum(world_suicide["Total suicides per 100k"]) / len(world_suicide["year"]), 2
)
avg_rate

11.83

## Top 10 countries with highest number of suicides per 100k

In [22]:
world_suicide_country = (
    data.groupby("country")
    .agg({"suicides/100k pop": "mean"})
    .rename(columns={"suicides/100k pop": "Suicide Rate", "country": "Country"})
    .sort_values(by=["Suicide Rate"], ascending=False)
    .reset_index()
)

world_suicide_country["Suicide Rate"] = round(
    world_suicide_country["Suicide Rate"] / 100, 2
)
world_suicide_country_top = world_suicide_country.head(10)
world_suicide_country_top

,country,Suicide Rate
0,Republic of Korea,0.46
1,Sri Lanka,0.35
2,Lithuania,0.34
3,Russian Federation,0.30
4,Hungary,0.29
5,Kazakhstan,0.27
6,Belarus,0.26
7,Latvia,0.25
8,Slovenia,0.24
9,Ukraine,0.23


In [23]:
## Avg suicide rate per country

avg_countries = round(
    sum(world_suicide_country["Suicide Rate"]) / len(world_suicide_country.country), 2
)
avg_countries
df2 = {"country": "All Countries", "Suicide Rate": 0.1}
df_world_country = world_suicide_country_top.append(df2, ignore_index=True)
df_world_country = df_world_country.sort_values(
    by=["Suicide Rate"], ascending=True
).reset_index()
df_world_country

,index,country,Suicide Rate
0,10,All Countries,0.10
1,9,Ukraine,0.23
2,8,Slovenia,0.24
3,7,Latvia,0.25
4,6,Belarus,0.26
5,5,Kazakhstan,0.27
6,4,Hungary,0.29
7,3,Russian Federation,0.30
8,2,Lithuania,0.34
9,1,Sri Lanka,0.35


In [24]:
listtop = world_suicide_country_top["country"].to_list()
listtop

['Republic of Korea',
 'Sri Lanka',
 'Lithuania',
 'Russian Federation',
 'Hungary',
 'Kazakhstan',
 'Belarus',
 'Latvia',
 'Slovenia',
 'Ukraine']

In [25]:
## top 10 countries compared to the world
country_pct = pd.DataFrame(
    {
        "Country": ["Rest of the world", "Top 10 countries"],
        "% Suicides": [
            round(
                sum(
                    world_suicide_country[~world_suicide_country.country.isin(listtop)][
                        "Suicide Rate"
                    ]
                )
                / sum(world_suicide_country["Suicide Rate"]),
                2,
            ),
            round(
                sum(world_suicide_country_top["Suicide Rate"])
                / sum(world_suicide_country["Suicide Rate"]),
                2,
            ),
        ],
    }
)

In [26]:
country_pct

,Country,% Suicides
0,Rest of the world,0.74
1,Top 10 countries,0.26


## Correlation between GDP & suicide rate

In [27]:
data.rename(
    columns={
        " gdp_for_year ($) ": "gdp_for_year",
        " gdp_per_capita ($) ": "gdp_per_capita",
        "gdp_per_capita ($)": "gdp_per_capita",
    },
    inplace=True,
)

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31756 entries, 0 to 31755
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            31756 non-null  object 
 1   year               31756 non-null  int64  
 2   sex                31756 non-null  object 
 3   age                31756 non-null  object 
 4   suicides_no        30556 non-null  float64
 5   population         31756 non-null  int64  
 6   suicides/100k pop  31756 non-null  float64
 7   country-year       31756 non-null  object 
 8   HDI for year       12300 non-null  float64
 9   gdp_for_year       31756 non-null  object 
 10  gdp_per_capita     31756 non-null  float64
 11  generation         31756 non-null  object 
dtypes: float64(4), int64(2), object(6)
memory usage: 2.9+ MB


In [29]:
data["gdp_for_year"] = data["gdp_for_year"].str.replace(",", "")

In [30]:
data["gdp_for_year"] = data["gdp_for_year"].astype(float)
data["gdp_for_year"] = data["gdp_for_year"].apply(lambda x: "%.3f" % x)
data_gdp = data.groupby(["country", "year"]).gdp_per_capita.mean()

In [31]:
data_gdp

country     year
Albania     1987     796.000000
            1988     769.000000
            1989     833.000000
            1992     251.000000
            1993     437.000000
                       ...     
Uzbekistan  2014    2309.000000
            2017    1916.764642
            2018    1597.068337
            2019    1784.009816
            2020    1750.697113
Name: gdp_per_capita, Length: 2649, dtype: float64

In [32]:
data.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year,gdp_per_capita,generation
0,Albania,1987,male,15-24 years,21.0,312900,6.71,Albania1987,NaN,2156624900.000,796.0,Generation X
1,Albania,1987,male,35-54 years,16.0,308000,5.19,Albania1987,NaN,2156624900.000,796.0,Silent
2,Albania,1987,female,15-24 years,14.0,289700,4.83,Albania1987,NaN,2156624900.000,796.0,Generation X
3,Albania,1987,male,75+ years,1.0,21800,4.59,Albania1987,NaN,2156624900.000,796.0,G.I. Generation
4,Albania,1987,male,25-34 years,9.0,274300,3.28,Albania1987,NaN,2156624900.000,796.0,Boomers


In [33]:
gdp_agg = (
    data.groupby(["country", "year"])
    .agg({"gdp_per_capita": "mean", "suicides/100k pop": "mean"})
    .reset_index()
)
gdp_agg.gdp_per_capita = round(gdp_agg.gdp_per_capita, 2)
gdp_agg["suicides/100k pop"] = round(gdp_agg["suicides/100k pop"], 2)

In [34]:
## Find the correlation coeff between gdp per capital and suicides/100k population
corr_try = {}
for country in listtop[:11]:
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        gdp_agg[gdp_agg.country == country]["gdp_per_capita"].values,
        gdp_agg[gdp_agg.country == country]["suicides/100k pop"].values,
    )
    corr_try[country] = float(r_value)
corr_try

{'Republic of Korea': 0.7251359169080739,
 'Sri Lanka': -0.8093109889384883,
 'Lithuania': -0.8271843111165436,
 'Russian Federation': -0.6997360583081011,
 'Hungary': -0.8555249238403445,
 'Kazakhstan': -0.5967405701525098,
 'Belarus': -0.6417361708586563,
 'Latvia': -0.84142717571815,
 'Slovenia': -0.7284791214984401,
 'Ukraine': -0.6809900309563642}

In [35]:
# corr_eff = {}
# for country in listtop[:11]:
#   slope, intercept, r_value, p_value, std_err = stats.linregress(
#      data_gdp[country].values, data_total[country].values
#  )
#         corr_eff[country] = float(r_value)

In [36]:
corr_eff_data = pd.DataFrame.from_dict(corr_try, orient="index")
corr_eff_data.reset_index(inplace=True)

In [37]:
corr_eff_data = corr_eff_data.rename(columns = {'index':'Country',0:'Correlation Coeff'})


In [38]:
corr_eff_data.Country.values

array(['Republic of Korea', 'Sri Lanka', 'Lithuania',
       'Russian Federation', 'Hungary', 'Kazakhstan', 'Belarus', 'Latvia',
       'Slovenia', 'Ukraine'], dtype=object)

In [39]:
gdp_agg.head()

,country,year,gdp_per_capita,suicides/100k pop
0,Albania,1987,796.0,2.65
1,Albania,1988,769.0,2.70
2,Albania,1989,833.0,2.78
3,Albania,1992,251.0,1.50
4,Albania,1993,437.0,2.71


In [40]:
country_top_gdp = gdp_agg[gdp_agg.country.isin(listtop)]
country_top_gdp = country_top_gdp.sort_values(by="year", ascending=True)


In [41]:
def do_regression(data):
    x = data['gdp_per_capita'].values.reshape(-1, 1)
    y = data['suicides/100k pop'].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(x, y)
    data['predictions'] = model.predict(x)
    return data


In [42]:
regression_chart = do_regression(gdp_agg)
regression_chart

,country,year,gdp_per_capita,suicides/100k pop,predictions
0,Albania,1987,796.00,2.65,11.670242
1,Albania,1988,769.00,2.70,11.670165
2,Albania,1989,833.00,2.78,11.670348
3,Albania,1992,251.00,1.50,11.668682
4,Albania,1993,437.00,2.71,11.669214
...,...,...,...,...,...
2644,Uzbekistan,2014,2309.00,7.09,11.674573
2645,Uzbekistan,2017,1916.76,0.62,11.673450
2646,Uzbekistan,2018,1597.07,0.58,11.672535
2647,Uzbekistan,2019,1784.01,0.58,11.673070


In [43]:
# initialize variables for selector
countries = ['All countries'] + listtop
selected_countryy = 'All countries'

# when the selected value of the country selector changes
# call this function
def update_regresion(state):
    if state.selected_countryy == 'All countries':
        temp_data = gdp_agg.copy()
    else:
        temp_data = gdp_agg[gdp_agg['country'] == state.selected_countryy]
    
    state.regression_chart = do_regression(temp_data)

#Gui(page).run(port=5000)


## Initialization

In [44]:
# by year
def initialize_case_evolution(data, selected_country="Republic of Korea"):
    # Aggregation of the dataframe to erase the regions that will not be used here
    data_country = (
        data.groupby(["country", "year"])
        .agg({"suicides/100k pop": "mean"})
        .rename(columns={"year": "Year", "suicides/100k pop": "Total suicides"})
        .reset_index()
    )
    # a country is selected, here Korea by default
    data_country = data_country.loc[data_country["country"] == selected_country]
    return data_country


data_country = initialize_case_evolution(data)

In [45]:
# gdp by year
def initialize_case_gdp(data, selected_country="Republic of Korea"):
    # Aggregation of the dataframe to erase the regions that will not be used here
    gdp_country = data[["country", "year", "gdp_per_capita"]]
    # a country is selected, here Korea by default
    gdp_country = gdp_country.loc[gdp_country["country"] == selected_country]
    gdp_country = gdp_country.sort_values(by="year", ascending=True)
    return gdp_country


gdp_country = initialize_case_gdp(country_top_gdp)

In [46]:
def on_change_country(state):
    # state contains all the Gui variables and this is through this state variable that we can update the Gui
    # update data_country_date with the right country (use initialize_case_evolution)
    print("Chosen country: ", state.selected_country)
    state.gdp_country = initialize_case_gdp(data, state.selected_country)
    state.data_country = initialize_case_evolution(data, state.selected_country)


#    state.gdp_suicide = filter_country(gdp_agg, state.selected_country)

In [47]:
# Initial selection
selected_country = "Republic of Korea"
selection = "Global Suicides by Years"
prop_bar = {"orientation": "h"}

# Format
layout = {"hovermode": "x"}
options = {"unselected": {"marker": {"opacity": 0.75}}}

# Pie
layoutpie = {
    "piecolorway": ["#38a7b8", "#14344b"],
}
pieage = {
    "piecolorway": ["#38a7b8", "#14344b", "#c8fafb", "#eef0ef", "#fbd956", "#bf5550"],
}

In [48]:
tryy = """<|toggle|theme|>
<|part|class_name=p3|

# IS SUICIDE RATE *INCREASING?*{: .color-primary}

Suicide, which involves intentionally ending one's life, is a complex and tragic phenomenon that is a leading cause of death worldwide. However, it is important to emphasize that suicide is not a solution to problems. Despite this 
fact, the distressing reality is that millions of people attempt suicide and hundreds of thousands die by suicide each year. 

## *Good News:*{: .color-primary} The Global Trend is Decreasing

<|{selection}|toggle|lov=Global Suicides by Years;Global Suicides by Genders;Global Suicides by Ages|> 


<global_suicides|part|render={selection=='Global Suicides by Years'}|
<|layout|gap=30px|columns=2 1|
### **Worldwide**{: .color-primary} Suicide (per 100k) by Years  <br/> <br/> \
<|{world_suicide}|chart|type=line|x=year|y=Total suicides per 100k|layout={layout}|height=100%|>  

<right_col|
<br/> <br/>
<avg_suicides|part|class_name=card text-center m2|
**11.83**

Average Suicides per 100K (1985-2015)
|avg_suicides>

<avg_10|part|class_name=card text-center m2|
**29.91**

Average Suicides per 100K (Top 10 Countries)
|avg_10>

### % of **Suicides**{: .color-primary} in top 10 Countries vs. Rest of The World  <br/> <br/>\
<|{country_pct}|chart|type=pie|values=% Suicides|labels=Country|layout={layoutpie}|>  
|right_col>
|>
|global_suicides> 

<gender|part|render={selection=='Global Suicides by Genders'}|
<|layout|gap=30px|columns=2 1|
<left_col|
### Worldwide Suicide (per 100k) by **Genders**{: .color-primary}
<|{world_suicide_sex}|chart|mode=lines|x=year|y[1]=suicides/100k pop_female|y[2]=suicides/100k pop_male|color[1]=#14344b|color[2]=#38a7b8|height=900px|>  
|left_col>



<right_col|
<br/> <br/>
<avg_suicides|part|class_name=card text-center m2|
**18.55**

Average Suicides per 100K of **Men**{: .color-primary}
|avg_suicides>

<avg_suicides|part|class_name=card text-center m2|
**5.12**

Average Suicides per 100K of **Women**{: .color-primary}
|avg_suicides>

### % of Suicides **Men vs. Women**{: .color-primary}  <br/> <br/>\
<|{gender_pct}|chart|type=pie|values=% Suicides|labels=Gender|layout={layoutpie}|>  
|right_col>
|>
|gender>

<ages|part|render={selection=='Global Suicides by Ages'}|

<|layout|gap=30px|columns=2 1|
### Worldwide Suicide (per 100k) by **Ages**{: .color-primary} <br/> <br/>\
<|{world_suicide_age}|chart|mode=lines|x=year|y[1]=suicides/100k pop_5-14 years|y[2]=suicides/100k pop_15-24 years|y[3]=suicides/100k pop_25-34 years|y[4]=suicides/100k pop_35-54 years|y[5]=suicides/100k pop_55-74 years|y[6]=suicides/100k pop_75+ years|color[1]=darkblue|color[2]=#4f6871|color[3]=#112635|color[4]=#bf5550|color[5]=#009681|color[6]=#ffd10c|height=900px|>


<right_col|
<br/> <br/>
<highest|part|class_name=card text-center m2|
**21.82**

Highest Avg Suicides per 100K: 75+ Age Group
|highest>

<lowest|part|class_name=card text-center m2|
**0.54**

Lowest Avg Suicides per 100K: 5-14 Age Group
|lowest>

###% of Suicides between **Age**{: .color-primary} Groups <br/> <br/> \
<|{age_pct}|chart|type=pie|values=% Suicides|labels=Age|layout={pieage}|>  

|right_col> 
|>

|ages>

## **Korea**{: .color-primary} Has The Highest Suicide Rate

<|{selected_country}|selector|lov={listtop}|dropdown|on_change=on_change_country|label=Selected Country|>

<|layout|gap=30px|columns=60 40|
### **Top 10**{: .color-primary} Countries with The Highest Suicide Rates  <br/> <br/>\
<|{df_world_country}|chart|type=bar|y=country|x=Suicide Rate|properties={prop_bar}|height=800px|>

<right_col|
### Country **Average**{: .color-primary} Suicide by Year <br/> <br/>\
<|{data_country}|chart|type=line|x=year|y=Total suicides|layout={layout}|height=400px|>

### **Gdp**{: .color-primary} per Capita by Year  <br/> <br/>\
<|{gdp_country}|chart|type=line|x=year|y=gdp_per_capita|layout={layout}|height=400px|>
|right_col>
|>

## The Suicide Rate is **Highly Correlated**{: .color-primary} to The Gdp per Capita of Top 10 Countries

<|{selected_countryy}|selector|lov={countries}|dropdown|on_change=update_regresion|label=Selected Country|>

<|layout|gap=30px|columns=1 1|
### **Correlation**{: .color-primary} Coeff. of GDP vs. Suicide Rates <br/> <br/>\
<|{corr_eff_data}|chart|type=bar|y=Correlation Coeff|x=Country|color=#14344b|height=600px|>

### **Regression**{: .color-primary} Plot of GDP per Capita & Suicide Rates  <br/> <br/>\
<|{regression_chart}|chart|x=gdp_per_capita|y[1]=suicides/100k pop|y[2]=predictions|mode[1]=markers|mode[2]=lines|height=600px|>
|>
|>
"""

In [50]:
import os

# Run Taipy GUI
gui = Gui(tryy, css_file="taipy/main.css", path_mapping={"taipy": "D:/Jupyter Notebook/taipy"})

gui.run(dark_mode=False, port=5007, path_mapping={"taipy": "D:/Jupyter Notebook/taipy"})

[2023-05-05 14:17:10,530][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-05-05 14:17:10,695][Taipy][INFO]  * Server starting on http://127.0.0.1:5007
